In [ ]:
import json
from collections import defaultdict

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[0] + box1[2], box2[0] + box2[2])
    y2 = min(box1[1] + box1[3], box2[1] + box2[3])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area if union_area > 0 else 0
    return iou

with open('/content/drive/MyDrive/4th_Semester/Capstone/Original_Size/instances_val_trashcan.json', 'r') as f:
    data = json.load(f)

with open('/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_val_gt/bbox_detections.json', 'r') as f:
    pred_data = json.load(f)

annotations_by_image_category = defaultdict(lambda: defaultdict(list))
for annotation in data['annotations']:
    annotations_by_image_category[annotation['image_id']][annotation['category_id']].append(annotation['bbox'])

results = []

for image in data['images']:
    image_id = image['id']
    image_name = image['file_name']

    ground_truth_annotations = annotations_by_image_category[image_id]

    predictions = [pred for pred in pred_data if pred['image_id'] == image_id]

    predictions_by_category = defaultdict(list)
    for pred in predictions:
        if pred['score'] > 0.15:
            predictions_by_category[pred['category_id']].append(pred['bbox'])

    iou_scores = {}

    for category_id, gt_boxes in ground_truth_annotations.items():
        category_iou_scores = []
        for gt_box in gt_boxes:
            for pred_box in predictions_by_category[category_id]:
                iou = calculate_iou(gt_box, pred_box)
                category_iou_scores.append(iou)
        max_iou = max(category_iou_scores) if category_iou_scores else 0
        iou_scores[category_id] = max_iou

    results.append({
        'image_id': image_id,
        'image_name': image_name,
        'iou_scores': iou_scores
    })

with open('/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_val_gt/iou_original.json', 'w') as f:
    json.dump(results, f, indent=4)


In [ ]:
from PIL import Image
import os

directory = "/content/drive/MyDrive/4th_Semester/Capstone/Original_Size/train"

count = 0
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        image_path = os.path.join(directory, filename)
        count+=1
        '''
        # Open the image using PIL
        with Image.open(image_path) as img:
            # Get the resolution of the image
            width, height = img.size

            # Check if resolution is not equal to 480x270
            if width != 480 or height != 270:
                print(f"Image: {filename}, Resolution: {width}x{height}")
'''
print(count)

6065


In [ ]:
import json
import csv

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def compare_iou(ground_truth, predicted):
    results = []
    for gt, pred in zip(ground_truth, predicted):
        if gt['image_id'] != pred['image_id']:
            raise ValueError("Image IDs don't match")

        image_id = gt['image_id']
        image_name = gt['image_name']
        gt_iou_scores = gt['iou_scores']
        pred_iou_scores = pred['iou_scores']

        label = 1 if any(pred_iou_scores[cat] > gt_iou_scores[cat] * 1.05 for cat in gt_iou_scores) else 0

        results.append({
            'image_id': image_id,
            'image_name': image_name,
            'label': label
        })

    return results

def save_to_csv(results, file_path):
    with open(file_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['image_id', 'image_name', 'label'])
        writer.writeheader()
        writer.writerows(results)

ground_truth = load_json('/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_train_gt/iou_original.json')
predicted = load_json('/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_train_p/iou_predicted.json')

results = compare_iou(ground_truth, predicted)

save_to_csv(results, '/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_train_p/comparison_results.csv')


In [ ]:
import csv

def count_labels(csv_file):
    count_ones = 0
    count_zeros = 0

    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            label = int(row['label'])
            if label == 1:
                count_ones += 1
            elif label == 0:
                count_zeros += 1

    return count_ones, count_zeros

csv_file = '/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_train_p/comparison_results.csv'

count_ones, count_zeros = count_labels(csv_file)

print("Number of ones:", count_ones)
print("Number of zeros:", count_zeros)


Number of ones: 329
Number of zeros: 5736
